In [11]:
import pandas as pd
import os
import time
from alocacao import alocar_voos
from vizinhancas.vnd import VND

def gap(solucao, solucao_otima):
    return ((solucao - solucao_otima) / solucao_otima) * 100 if solucao_otima > 0 else -1.0

def gera_tabela_gap(df_otimo, caminho_saida='gap.csv'):
    # Verifica se o arquivo já existe para evitar recriar o cabeçalho
    if os.path.exists(caminho_saida):
        df_gap = pd.read_csv(caminho_saida)
    else:
        df_gap = pd.DataFrame()

    for nome_arquivo in sorted(os.listdir('instancias_teste')):
            if nome_arquivo.endswith(".txt"):
                caminho = os.path.join('instancias_teste', nome_arquivo)
                print(f"Processando arquivo: {caminho}")
                
                otimo_val = df_otimo.loc[df_otimo['Instancia'] == nome_arquivo, 'Otimo'].values[0]

                inicio_guloso = time.time()
                pistas_gulosas, custo_guloso, dados = alocar_voos(caminho)
                fim_guloso = time.time()
                gap_guloso = gap(custo_guloso, df_otimo.loc[df_otimo['Instancia'] == nome_arquivo, 'Otimo'].values[0])


                # Executa o VND
                inicio_vnd = time.time()
                pistas_vnd, custo_vnd = VND(pistas_gulosas, custo_guloso, dados)
                fim_vnd = time.time()
                gap_vnd = gap(custo_vnd, df_otimo.loc[df_otimo['Instancia'] == nome_arquivo, 'Otimo'].values[0])


                nova_linha = pd.DataFrame([{
                'Instancia': nome_arquivo,
                'Otimo': otimo_val,
                'Solucao (HC)': custo_guloso,
                'Tempo (HC)': f"{fim_guloso - inicio_guloso:.4f} segundos",
                'Gap (HC)': round(gap_guloso, 2),
                'Solucao (VND)': custo_vnd,
                'Tempo (VND)': f"{fim_vnd - inicio_vnd:.4f} segundos",
                'Gap (VND)': round(gap_vnd, 2),
                }])
                df_gap = pd.concat([df_gap, nova_linha], ignore_index=True)

    df_gap.to_csv('gap.csv',sep=',',index=False)


otimo = pd.DataFrame({
    'Instancia' : ['n3m10A.txt','n3m10B.txt','n3m10C.txt','n3m10D.txt',
                   'n3m10E.txt','n3m20A.txt','n3m20B.txt','n3m20C.txt',
                   'n3m20D.txt','n3m20E.txt','n3m40A.txt','n3m40B.txt',
                   'n3m40C.txt','n3m40D.txt','n3m40E.txt','n5m50A.txt',
                   'n5m50B.txt','n5m50C.txt','n5m50D.txt','n5m50E.txt'],
    'Otimo' : [7483,1277,2088,322,3343,8280,1820,855,4357,3798,
              112,880,1962,263,1192,0,0,0,0,0]
})

gera_tabela_gap(otimo)


Processando arquivo: instancias_teste\n3m10A.txt
Pista 1
  QUICK8  (Início: 12, Multa: 0)
  QUICK10  (Início: 81, Multa: 3200)
  QUICK1  (Início: 120, Multa: 3648)
  QUICK7  (Início: 172, Multa: 8774)
Pista 2
  QUICK2  (Início: 15, Multa: 0)
  QUICK6  (Início: 70, Multa: 320)
  QUICK4  (Início: 111, Multa: 3450)
Pista 3
  QUICK5  (Início: 29, Multa: 0)
  QUICK3  (Início: 69, Multa: 2800)
  QUICK9  (Início: 137, Multa: 1748)

Custo total da alocação: 23940


Custo total da alocação VND: 7483

Processando arquivo: instancias_teste\n3m10B.txt
Pista 1
  QUICK9  (Início: 6, Multa: 0)
  QUICK8  (Início: 30, Multa: 0)
  QUICK1  (Início: 98, Multa: 750)
Pista 2
  QUICK2  (Início: 17, Multa: 0)
  QUICK4  (Início: 79, Multa: 2028)
  QUICK7  (Início: 115, Multa: 660)
Pista 3
  QUICK10  (Início: 21, Multa: 0)
  QUICK5  (Início: 36, Multa: 0)
  QUICK6  (Início: 68, Multa: 405)
  QUICK3  (Início: 91, Multa: 1716)

Custo total da alocação: 5559


Custo total da alocação VND: 1277

Processando arquivo

In [14]:
df_gap = pd.read_csv('gap.csv')
display(df_gap)

,Instancia,Otimo,Solucao (HC),Tempo (HC),Gap (HC),Solucao (VND),Tempo (VND),Gap (VND)
0,n3m10A.txt,7483,23940,0.0003 segundos,219.93,7483,0.0348 segundos,0.00
1,n3m10B.txt,1277,5559,0.0004 segundos,335.32,1277,0.0278 segundos,0.00
2,n3m10C.txt,2088,10301,0.0004 segundos,393.34,2150,0.0416 segundos,2.97
3,n3m10D.txt,322,5124,0.0005 segundos,1491.30,322,0.0396 segundos,0.00
4,n3m10E.txt,3343,6920,0.0004 segundos,107.00,3452,0.0281 segundos,3.26
5,n3m20A.txt,8280,88820,0.0005 segundos,972.71,38393,0.6075 segundos,363.68
6,n3m20B.txt,1820,66010,0.0005 segundos,3526.92,23354,0.2864 segundos,1183.19
7,n3m20C.txt,855,33006,0.0005 segundos,3760.35,8293,0.2691 segundos,869.94
8,n3m20D.txt,4357,16477,0.0005 segundos,278.17,4785,0.3029 segundos,9.82
9,n3m20E.txt,3798,33326,0.0005 segundos,777.46,3827,0.2962 segundos,0.76
